In [13]:
from dotenv import load_dotenv
load_dotenv()
import os
import pinecone
from openai import OpenAI

In [18]:
pc_api_key = os.getenv("PINECONE_API_KEY")
pc=pinecone.Pinecone(api_key=pc_api_key)
pc.create_index(name="rag", dimension=1536, metric="cosine", spec=pinecone.ServerlessSpec(cloud="aws", region="us-east-1"))


In [25]:
import json
data = json.load(open("reviews.json"))
data['reviews']


[{'professor': 'Dr. Jane Smith',
  'review': 'Excellent teacher who explains complex concepts clearly. Assignments are challenging but fair.',
  'subject': 'Computer Science',
  'stars': 5},
 {'professor': 'Prof. John Doe',
  'review': 'Knowledgeable but lectures can be hard to follow. Office hours are very helpful.',
  'subject': 'Mathematics',
  'stars': 4},
 {'professor': 'Dr. Emily Chen',
  'review': 'Passion for chemistry is contagious. Tough course, but she makes it interesting.',
  'subject': 'Chemistry',
  'stars': 4},
 {'professor': 'Prof. Michael Johnson',
  'review': 'Brings history to life with storytelling. Exams are straightforward if you attend lectures.',
  'subject': 'History',
  'stars': 5},
 {'professor': 'Dr. Sarah Lee',
  'review': 'At the cutting edge of AI research. Course is demanding but rewarding.',
  'subject': 'Artificial Intelligence',
  'stars': 4},
 {'professor': 'Prof. Robert Brown',
  'review': 'Lectures can be dry, but the content is interesting. Easy 

In [26]:
api_key = os.getenv('OPENAI_API_KEY')
processed_data = []
client = OpenAI(api_key=api_key)

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'], 
        model="text-embedding-3-small",

    )
    embedding = response.data[0].embedding
    processed_data.append({"values": embedding, 
                           "id": review['professor'], 
                           "metadata": {
                               "review": review["review"], 
                               "subject": review['subject'],
                               "stars" : review["stars"]}})


In [27]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)


{'upserted_count': 39}

In [28]:
#Index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 39}},
 'total_vector_count': 39}